# RACMO

"The surface mass balance (SMB) is from RACMO2.3p2, averaged for the years 1979-2011 [van Wessem
et al., 2018], with an accuracy of 7% for the time-average fields."

- **rlat = y**
- **rlon = x**

(super-res conda has xarray)

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr

In [27]:
path_server = "/home/kim/data/zenodo_7760491_RACMO2-3p2/smb_monthlyS_ANT27_ERA5-3H_RACMO2.3p2_197901_202212.nc"
racmo = xr.open_dataset(path_server)

## Download BedMachine for grid

In [26]:
target_grid = pd.read_csv("data/BedMachine_500mgrid_Byrd.csv")

# Slice time according to BedMachine

In [28]:
# racmo.time.values
time_min = '1979-01-16T00:00:00.000000000'
time_max = '2011-12-16T00:00:00.000000000'

# Inclusive slice
racmo_1979_2011 = racmo.sel(time = slice(time_min, time_max))

## Take temporal average 

In [29]:
# smb_average = racmo.mean(dim = "time").smb
# units: kg m-2
smb_average = racmo_1979_2011.mean(dim = "time").smb

In [38]:
# adjust unit to rlat and rlon
div = 100000
y_min = (- 800000) / div
y_max = - 1000000 / div
x_min = 350000 / div
x_max = (549500 + 500) / div

In [46]:
selection = smb_average.sel(rlat = slice(y_max, y_min),
                     rlon = slice(x_min, x_max))

selection

<xarray.DataArray 'smb' (height: 1, rlat: 9, rlon: 9)>
array([[[ 8.959297 ,  9.470294 , 10.222874 , 11.351175 , 13.71516  ,
         16.925936 , 24.971292 , 38.490566 , 23.435095 ],
        [ 8.68002  ,  9.199692 ,  9.9171095, 11.04783  , 13.524281 ,
         16.692762 , 18.666994 , 24.980795 , 33.695576 ],
        [ 8.476059 ,  8.984914 ,  9.842887 , 10.829416 , 13.22918  ,
         15.525223 , 15.752038 , 13.162089 , 18.488867 ],
        [ 8.445755 ,  8.978716 ,  9.85797  , 11.077313 , 12.659625 ,
         14.482186 , 14.305769 , 11.194419 , 31.24092  ],
        [ 8.555285 ,  9.232018 , 10.174193 , 11.789454 , 13.018149 ,
         13.113815 , 10.213007 ,  9.391708 , 16.667957 ],
        [ 8.7882805,  9.727531 , 11.004846 , 12.588956 , 14.478003 ,
         14.018339 , 11.719926 ,  2.4967349,  7.418627 ],
        [ 9.147995 , 10.3307085, 11.928703 , 13.5987215, 15.54632  ,
         11.194285 , 16.724356 , 13.325873 ,  8.509171 ],
        [ 9.851295 , 11.344387 , 12.873877 , 14.040842 , 15.81101  ,
         17.117851 ,  8.138181 , 20.757406 ,  8.613059 ],
        [10.992976 , 12.894165 , 13.982051 , 13.67685  , 13.669542 ,
         14.966661 ,  4.8912196,  4.0954213,  4.8525333]]], dtype=float32)
Coordinates:
  * height   (height) float64 0.0
    lat      (rlat, rlon) float64 ...
    lon      (rlat, rlon) float64 ...
  * rlat     (rlat) float64 -10.0 -9.75 -9.5 -9.25 -9.0 -8.75 -8.5 -8.25 -8.0
  * rlon     (rlon) float64 3.5 3.75 4.0 4.25 4.5 4.75 5.0 5.25 5.5

In [48]:
np.meshgrid(selection.rlat, selection.rlon)
# Inverse order to below

[array([[-10.  ,  -9.75,  -9.5 ,  -9.25,  -9.  ,  -8.75,  -8.5 ,  -8.25,
          -8.  ],
        [-10.  ,  -9.75,  -9.5 ,  -9.25,  -9.  ,  -8.75,  -8.5 ,  -8.25,
          -8.  ],
        [-10.  ,  -9.75,  -9.5 ,  -9.25,  -9.  ,  -8.75,  -8.5 ,  -8.25,
          -8.  ],
        [-10.  ,  -9.75,  -9.5 ,  -9.25,  -9.  ,  -8.75,  -8.5 ,  -8.25,
          -8.  ],
        [-10.  ,  -9.75,  -9.5 ,  -9.25,  -9.  ,  -8.75,  -8.5 ,  -8.25,
          -8.  ],
        [-10.  ,  -9.75,  -9.5 ,  -9.25,  -9.  ,  -8.75,  -8.5 ,  -8.25,
          -8.  ],
        [-10.  ,  -9.75,  -9.5 ,  -9.25,  -9.  ,  -8.75,  -8.5 ,  -8.25,
          -8.  ],
        [-10.  ,  -9.75,  -9.5 ,  -9.25,  -9.  ,  -8.75,  -8.5 ,  -8.25,
          -8.  ],
        [-10.  ,  -9.75,  -9.5 ,  -9.25,  -9.  ,  -8.75,  -8.5 ,  -8.25,
          -8.  ]]),
 array([[3.5 , 3.5 , 3.5 , 3.5 , 3.5 , 3.5 , 3.5 , 3.5 , 3.5 ],
        [3.75, 3.75, 3.75, 3.75, 3.75, 3.75, 3.75, 3.75, 3.75],
        [4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 

# only 8 x 8 values because grid is 27 km 

We can crop a litte larger and interpolate on target grid res.

In [41]:
# different order than the ones before
smb_interpolated = smb_average.interp(rlat = (target_grid["y"]/div), rlon = (target_grid["x"]/div), method = "cubic")
smb_interpolated

<xarray.DataArray 'smb' (height: 1, rlat: 400, rlon: 400)>
array([[[10.968796  , 11.00147401, 11.03468513, ...,  4.8585601 ,
          4.88091468,  4.90493323],
        [10.9445821 , 10.97709593, 11.0101282 , ...,  4.94998643,
          4.96812679,  4.98803032],
        [10.92034   , 10.95269125, 10.98554687, ...,  5.04317747,
          5.05679339,  5.07228265],
        ...,
        [ 8.94740918,  8.95548755,  8.96364671, ..., 24.85664328,
         24.49525371, 24.13893211],
        [ 8.95334709,  8.96139633,  8.96952326, ..., 24.68539696,
         24.31912784, 23.95849893],
        [ 8.95929718,  8.96731595,  8.9754091 , ..., 24.52891753,
         24.1579241 , 23.7931272 ]]])
Coordinates:
  * height   (height) float64 0.0
    lat      (rlat, rlon) float64 -81.27 -81.27 -81.26 ... -78.61 -78.61 -78.6
    lon      (rlat, rlon) float64 166.5 166.5 166.5 166.4 ... 161.5 161.5 161.5
  * rlat     (rlat) float64 -8.005 -8.01 -8.015 -8.02 ... -9.99 -9.995 -10.0
  * rlon     (rlon) float64 3.5 3.505 3.51 3.515 3.52 ... 5.48 5.485 5.49 5.495

In [43]:
smb_interpolated.values

array([[[10.968796  , 11.00147401, 11.03468513, ...,  4.8585601 ,
          4.88091468,  4.90493323],
        [10.9445821 , 10.97709593, 11.0101282 , ...,  4.94998643,
          4.96812679,  4.98803032],
        [10.92034   , 10.95269125, 10.98554687, ...,  5.04317747,
          5.05679339,  5.07228265],
        ...,
        [ 8.94740918,  8.95548755,  8.96364671, ..., 24.85664328,
         24.49525371, 24.13893211],
        [ 8.95334709,  8.96139633,  8.96952326, ..., 24.68539696,
         24.31912784, 23.95849893],
        [ 8.95929718,  8.96731595,  8.9754091 , ..., 24.52891753,
         24.1579241 , 23.7931272 ]]])

In [49]:
smb_interpolated.rlat

# rlat rlon order
yy, xx = np.meshgrid(smb_interpolated.rlat, smb_interpolated.rlon)